# Scenario 3: SCATTERED SPIDER Intrusion Analysis
### Description:
An organization becomes the victim of a targeted attack by the SCATTERED SPIDER threat group. The incident begins with social engineering tactics used to compromise the IT help desk and obtain credentials for initial access. Over time, the attackers escalate their privileges, establish persistence, and move laterally within the network. Eventually, BlackCat ransomware is deployed, locking down critical systems and exfiltrating sensitive customer data. The stolen data is used for extortion, threatening to release it unless a ransom is paid.

## Attributes to Include:
- Phone number used to call IT help desk
- C2 server domain and IP
- Legitimate software names and file hashes
- Malware name and file hashes
- Ransomware name and file hashes

### Indicators:
| Description                       | Category         | Type         | Value                                                            |
|-----------------------------------|------------------|--------------|------------------------------------------------------------------|
| Phishing Phone Number             | Person           | phone-number | +1-555-234-5678                                                  | 
| C2 Server Domain                  | Network activity | domain       | secure-update-check.com                                          |
| C2 Server IP Address              | Network activity | ip-dst       | 198.51.100.42                                                    |
| Legitimate Software File Name     | Network activity | filename     | AnyDesk.exe                                                      |
| Legitimate Software Hash (SHA256) | Network activity | sha256       | 6a2b5c7f8d14e3b9f5e9e1246a89e26b5e91b7120b34187dcdfb6c78f4ef5c22 |
| Malware File Name                 | Network activity | filename     | beacon.exe                                                       |
| Malware Hash (SHA256)             | Network activity | sha256       | 3e8d1c12a0f9b7e5bcb94e2ad10e53a4a3d2e2bdb4e09f2e5a8b732fbb4c9c66 |
| Ransomware File Name              | Network activity | filename     | bc_ransomware.exe                                                |
| Ransomware Hash (SHA256)          | Network activity | sha256       | b1d9b9f5f8c69e7a8b1d24c3c5d6e4b9a3c9c5f7d6e8f8b9e7f7d4c8b1e9a3f6 |


## Galaxy Clusters to Include:
### 1. Attack Pattern
 - Valid Accounts (Cloud Accounts):
   - Technique ID: T1078.004
   - Description: The attackers used valid credentials obtained through social engineering of the IT help desk to gain initial access.
 - Phishing (Spearphishing Voice):
   - Technique ID: T1566.004
   - Description: The attackers used social engineering to deceive the IT help desk into resetting credentials for initial access.
 - Remote Access Software:
   - Technique ID: T1219
   - Description: Legitimate remote access software like AnyDesk was used by the attackers to maintain persistence within the compromised environment.
 - Application Layer Protocol (Web Protocols):
   - Technique ID: T1071.001
   - Description: The attackers communicated with the C2 server using a domain (secure-update-check.com) over HTTP/HTTPS.
 - Data Encrypted for Impact
   - Technique ID: T1486
   - Description: BlackCat ransomware encrypted files on the compromised systems to deny access to key data and demand a ransom.
 - Obfuscated Files or Information
   - Technique ID: T1027
   - Description: The attackers likely used obfuscated files (e.g., Cobalt Strike beacon) to avoid detection while moving laterally in the network.

### 2. Malpedia:
 - Cobalt Strike
   - Description: Attribution to the specific malware
 - BlackCat
   - Description: Attribution to the specific ransomware

### 3. RH-ISAC Threat Actors
- SCATTERED SPIDER
  - Description: Attribution to the specific threat actor group


# Initialize environment
This section initializes the playbook environment and loads the required Python libraries. The credentials for MISP (authkey) is loaded from the file `keys.py` in the directory **vault**. A PyMISP object is created to interact with MISP and the active MISP server is displayed. By printing out the server name you know that it's possible to connect to MISP. In case of a problem, PyMISP will indicate the error with `PyMISPError: Unable to connect to MISP`.

The contents of the `keys.py` file should contain at least:
```
misp_url = "<MISP URL>"             # The URL to our MISP server
misp_key = "<MISP Authkey>"         # The MISP authkey
misp_verifycert = <True or False>   # Indicate if PyMISP should attempt to verify the certificate or ignore errors

In [ ]:
from vault.keys import *

from pymisp import PyMISP, MISPEvent, MISPObject


if not misp_verifycert:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
print("The \033[92mPython libraries\033[90m are loaded and the \033[92mcredentials\033[90m are read from the keys file.")

# Create the PyMISP object
misp = PyMISP(misp_url, misp_key, misp_verifycert)
print("I will use the MISP server \033[92m{}\033[90m for this playbook.\n\n".format(misp_url))


# Initialize MISP Event
This cell will initialize the MISP Event and attach the Event metadata.

In [ ]:
event = MISPEvent()
event.info = <Title of MISP Event>

## Threat Level
# 1 - High means sophisticated APT malware or 0day attack
# 2 - Medium means APT malware
# 3 - Low means mass malware
# 4 - Undefine

event.threat_level_id = <ID>


## Analysis
# 0 - Initial means Event has just been created and is in an initial state
# 1 - Ongoing means the analysis is still ongoing
# 2 - Completed means the Event creator considers the analysis complete

event.analysis = <ID>


## Distribution 
# 0 - Your organization means only members of your organization on this server will be able to see this Event
# 1 - This community only means organizations that are a part of this community server will be able to see this Event
# 2 - Connected communities means organizations that are a part of this community server or a part of a connected (synced) community MISP server will be able to see the event

event.distribution = <ID>

# Initialize Indicator Variables
This cell will initialize the indicator variables we will use to add Attributes to the Event.

In [ ]:
# Copy the indicators from the table above into the appropriate indicator variables

vishing_number = ""
c2_domain = ""
c2_ip = ""
legit_filename = ""
legit_sha256 = ""
malware_filename = ""
malware_sha256 = ""
ransomware_filename = ""
ransomware_sha256 = ""

# Add Attributes to the Event
This cell will add attributes to the Event.

In [ ]:
event.add_attribute(type="phone-number", value=vishing_number, category="Person")

# For added context, you can add a comment attribute that provides a brief description of the intel share
description = """
An organization becomes the victim of a targeted attack by the 
SCATTERED SPIDER threat group. The incident begins with social 
engineering tactics used to compromise the IT help desk and obtain 
credentials for initial access. Over time, the attackers escalate 
their privileges, establish persistence, and move laterally within 
the network. Eventually, BlackCat ransomware is deployed, locking 
down critical systems and exfiltrating sensitive customer data. 
The stolen data is used for extortion, threatening to release it 
unless a ransom is paid.
"""
event.add_attribute(type="comment", value=description)

# Initialize and add Attributes to MISP Object
This cell will initialize and add attributes to a MISP Object.

In [ ]:
legit_object = MISPObject(name="file")
legit_object.comment = "Legitimate software observed using during intrusion"

legit_object.add_attribute(object_relation="filename", simple_value=legit_filename)
legit_object.add_attribute(object_relation="sha256", simple_value=legit_sha256)


malware_object = MISPObject(name="file")
malware_object.comment = "Cobalt Strike beacon"

malware_object.add_attribute(object_relation="filename", simple_value=malware_filename)
malware_object.add_attribute(object_relation="sha256", simple_value=malware_sha256)


ransomware_object = MISPObject(name="file")
ransomware_object.comment = "BlackCat sample"

ransomware_object.add_attribute(object_relation="filename", simple_value=ransomware_filename)
ransomware_object.add_attribute(object_relation="sha256", simple_value=ransomware_sha256)


domain_ip_object = MISPObject(name="domain-ip")
domain_ip_object.comment = "Cobalt Strike C2"

domain_ip_object.add_attribute(object_relation="domain", simple_value=c2_domain)
domain_ip_object.add_attribute(object_relation="ip", simple_value=c2_ip)


# Add Objects to Event
event.add_object(legit_object)
event.add_object(malware_object)
event.add_object(ransomware_object)
event.add_object(domain_ip_object)

# Contextualize the Event with Galaxy Clusters
This cell will add context to the Event by utilizing galaxy cluster tags.

In [ ]:
# Attack Pattern
event.add_tag('misp-galaxy:mitre-attack-pattern="Cloud Accounts - T1078.004"')
event.add_tag('misp-galaxy:mitre-attack-pattern="Spearphishing Voice - T1566.004"')
event.add_tag('misp-galaxy:mitre-attack-pattern="Remote Access Software - T1219"')
event.add_tag('misp-galaxy:mitre-attack-pattern="Web Protocols - T1071.001"')
event.add_tag('misp-galaxy:mitre-attack-pattern="Data Encrypted for Impact - T1486"')
event.add_tag('	misp-galaxy:mitre-attack-pattern="Obfuscated Files or Information - T1027"')

# Malpedia Galaxy
event.add_tag('misp-galaxy:malpedia="BlackCat (Windows)"')

# RH-ISAC Threat Actors
event.add_tag('misp-galaxy:rhisac-threat-actors="SCATTERED SPIDER"')

# Publish and Share the Event
This cell will publish and share the event to the MISP server.

In [ ]:
event.publish()

misp.add_event(event)